In [3]:
#Libraries
import requests
import numpy as np
import pandas as pd
import json
import time
import pymsteams
import pyshorteners as ps
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.support.select import Select
import warnings
warnings.filterwarnings("ignore")

#Web Connection
driver = webdriver.Firefox()
#Minimize browser so that it does not jump on screen during script execution
driver.minimize_window()
url_begin= 'https://www.pap.hacienda.gob.es/bdnstrans/GE/es/convocatorias'
driver.get(url_begin)
title_sec = driver.title
url_sec = driver.current_url
#Select key words in title
date = driver.find_element_by_xpath('//*[@id="titulo"]')
date.clear()
key_words = 'innovación'
date.send_keys(key_words)


#Select Region options
region = Select(driver.find_element_by_xpath('//*[@id="regionalizacion"]'))
# select by visible text
#select.select_by_visible_text('xxxx')
# select by value 
region.select_by_value('1')
region.select_by_value('25')
#Select date options
date = driver.find_element_by_xpath('//*[@id="fecDesde"]')
date.clear()
date.send_keys('01/10/2022')
#Select search button
driver.find_element_by_xpath('/html/body/article/section[1]/div[2]/form/div[11]/ul/li[1]/button').click()
#Wait for the search to load
driver.implicitly_wait(15)

In [5]:
#Select the html code from the lookup table
tabla = driver.find_element_by_xpath('//*[@id="grid"]')

#Treat table html with BeautifulSoup
soup = BS(tabla.get_attribute('innerHTML'),"html.parser")

#We extract the data from the table in a dictionary
data = []
rows = soup.find_all('tr')[1:]
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    #data.append([ele for ele in cols if ele])
    data.append([ele for ele in cols]) #Eliminamos if ele para que tenga en cuenta los campos vacíos

In [17]:
#create df from dictionary
columns = ['Number','BD_Code','MRR','Authority','Organization','Depart','Date','Title', 'Title2', 'link', 'cod_desc']
df = pd.DataFrame(data)
df.columns = columns
#Keep only the columns with information necessary to generate the alarm
df = df[['BD_Code','MRR','Authority','Organization','Date','Title', 'link']]
#we bind to that the Column Codigo_BD is string type
df['BD_Code'] = df['BD_Code'].astype('string')
#We put the title in minuscule so that it occupies less in the notification of the Teams
df['Title'] = df['Title'].str.lower()
#Add column with the link to the call is necesary add BD_Code to the url
df['link'] = df['BD_Code'].apply(lambda x: url_sec[:-1]+'/'+str(x))

In [19]:
#Shortened the urls of the 'link' field so that they are read better in Teams with pyshorteners library
s = ps.Shortener()
df['link'] = df['link'].apply(lambda x: s.chilpit.short(x))

In [20]:
df

,BD_Code,MRR,Authority,Organization,Date,Title,link
0,658856,NO,OTROS,UNIVERSIDAD DE ALCALÁ,14/11/2022,ayudas para la realización de proyectos colabo...,http://chilp.it/8dcf402
1,658236,NO,ZARAGOZA,AYUNTAMIENTO DE ZARAGOZA,10/11/2022,convocatoria pública de subvenciones para la r...,http://chilp.it/32808f7
2,657285,SI,ESTADO,MINISTERIO DE CIENCIA E INNOVACIÓN,04/11/2022,resolución de 3 de noviembre de 2022 de la dir...,http://chilp.it/b36d7e7
3,653852,NO,ESTADO,MINISTERIO DE CIENCIA E INNOVACIÓN,17/10/2022,resolución de 18 de octubre de 2022 de la pres...,http://chilp.it/e5d508d


In [ ]:
#Generate a df only with the last and not repeated records to send the alarm
#Recover the old df of the previous revision
df_old = pd.read_csv(r'C:\Users\username\yourath\Review_calls_innovation.csv')
df_old['BD_Code'] = df_old['BD_Code'].astype('string')

#Compare two dfs and delete duplicate records
df_diff = pd.concat([df,df_old]).reset_index(drop=True).drop_duplicates(subset=['BD_Code'], keep=False)
#Select the fields that will appear in the Teams notification
df_diff = df_diff[['BD_Code','Authority','Organization', 'Title','link']]

#We save the extracted df again for in the next iteration compare with this
df.to_csv(r'C:\Users\username\yourath\Review_calls_innovation.csv', index=False)

In [ ]:
#Notification function with the pymsteams library
##generate a template of the message that will be sent to Teams
def mymessageTeams (df_diff, title_sec, url_sec):
    import pymsteams
    myTeamsMessage = pymsteams.connectorcard("url_connection_in_your_Teams_channel")
    #Texto inicial
    myTeamsMessage.text("@alerts | Scouting public Calls in spain App by JLF")
    # create the section
    myMessageSection = pymsteams.cardsection()
    # Section Title
    #myMessageSection.title(title_sec)
    # Activity Elements
    myMessageSection.activityTitle(title_sec)
    myMessageSection.activitySubtitle(url_sec)
    myMessageSection.activityImage("url_your_logo_image_of_Teams_channel.jpeg")
    # Section Text
    myMessageSection.text(df_diff.to_html())
    # Add your section to the connector card object before sending
    myTeamsMessage.addSection(myMessageSection)

    return myTeamsMessage.send()

In [ ]:
#Send notification if there are new records regarding the last csv saved
print("Scouting: BDNS | Spanish Grants Database")
#Decision loop send notification
if len(df_diff) != 0:
    print('Notified to Teams')
    mymessageTeams (df_diff, title_sec, url_sec)
    driver.quit()
else:
    print('No new notifications')
    driver.quit()
    pass